In [1]:
from eda_import import *

In [2]:
from utils.cluster import cluster_clv_sites

In [5]:
# UHRC1
input_bed = '/projects/btl/zxue/tasrkleat-TCGA-results/tasrkleat-TCGA-analysis-scripts/benchmark-kleat/UHR/C1/polyA-Seq/GSM747475_human_maqc-UHR1.sites.clustered.hg19.bed.gz'

In [6]:
outdir = os.path.dirname(input_bed)

df_polya = pd.read_csv(
    input_bed, header=None, sep='\t',
    names=['seqname', 'clv', 'clv1', 'hexamer_str', 'unknown', 'strand'])

In [7]:
df_polya.head()

,seqname,clv,clv1,hexamer_str,unknown,strand
0,chr1,16442,16443,6.264|CATAAA,2,-
1,chr1,85379,85380,7.387|none,2,-
2,chr1,89297,89298,6.583|AGTAAA,3,-
3,chr1,123727,123728,10.25|AATAAA,1,-
4,chr1,564950,564951,5.730|none,1,-


# Filter against the coordinates of 114 genes

In [8]:
ndf_polya = df_polya.drop(['clv1', 'unknown'], axis=1)

In [10]:
df_gene_coords = pd.read_csv('./114_gene_coordinates.csv')

In [13]:
df_gene_coords.shape

(114, 16)

In [14]:
df_gene_coords.head(2)

,seqname,source,feature,start,end,score,strand,frame,len,transcript_id,gene_id,gene_name,gene_source,transcript_source,is_cds_end_NF,is_cds_start_NF
0,chr1,protein_coding,gene,40361098,40367928,.,-,.,6831,NaN,ENSG00000116990,MYCL,ensembl_havana,NaN,False,False
1,chr1,protein_coding,gene,51426417,51440305,.,+,.,13889,NaN,ENSG00000123080,CDKN2C,ensembl_havana,NaN,False,False


In [15]:
df_gene_coords.seqname.unique()

array(['chr1', 'chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'X', 'chr8',
       'chr9', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15',
       'chr16', 'chr17', 'chr18', 'chr20', 'chr19', 'chr22'], dtype=object)

In [16]:
COORDS_DD = df_gene_coords.groupby(['seqname', 'strand']).apply(lambda g: g[['start', 'end']].values).to_dict()
def is_from_target_gene(row):
    key = (row.seqname, row.strand)
    coord_list = COORDS_DD.get(key)
    if coord_list is None:
        return False
    
    valid = False
    for (beg, end) in coord_list:
        if row.clv >= beg and row.clv <= end:
            valid = True
            break
    return valid

In [17]:
%time ndf_polya['within_target_gene'] = ndf_polya.apply(is_from_target_gene, axis=1)

CPU times: user 19.7 s, sys: 0 ns, total: 19.7 s
Wall time: 19.6 s


In [18]:
ndf_polya.within_target_gene.value_counts()

False    103277
True     619   
Name: within_target_gene, dtype: int64

In [19]:
odf_polya = ndf_polya.query('within_target_gene == True')

In [20]:
odf_polya.head(2)

,seqname,clv,hexamer_str,strand,within_target_gene
1229,chr1,40361100,5.386|AATAAA,-,True
1230,chr1,40361235,7.453|none,-,True


In [21]:
odf_polya.shape

(619, 5)

# Cluster doesn't change the results 

polyA-seq results could've been clustered already

In [24]:
def cluster_to_stable(input_df):
    _df = input_df.copy()
    for i in range(2):
        print('{0} clustering...'.format(i))
        _df = _df.groupby(['seqname', 'strand']).apply(cluster_clv_sites, 20).reset_index(drop=True)
    return _df

cutoff = 20 # bp
%time pdf_polya = cluster_to_stable(odf_polya)

assert pdf_polya[['seqname', 'strand', 'clv']].drop_duplicates().shape[0] == 619

0 clustering...
1 clustering...
CPU times: user 1.58 s, sys: 8.88 ms, total: 1.59 s
Wall time: 1.58 s


So clustering polya-seq does not change the results!

In [25]:
odf_polya.to_csv(os.path.join(outdir, 'polyA-Seq-truth-114-genes.csv'), index=False)